In [2]:
from pathlib import Path

import nibabel as nib
import pandas as pd

output_dir = Path("/media/storage/yalab-dev/voxelops/")
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
import numpy as np
from templateflow import api as tf

NAMING_SCHEME = "Schaefer2018N{n_parcels}n{n_networks}Tian2020S{scale}"


def load_lut(n_schaefer: int | str, scale: int | str):
    n_4s = int(n_schaefer) + 56
    atlas_4s = output_dir / "atlases" / f"atlas-4S{n_4s}Parcels"
    parcels_4s = atlas_4s / f"atlas-4S{n_4s}Parcels_dseg.tsv"
    parcels_4s = pd.read_csv(parcels_4s, sep="\t")
    parcels_4s["atlas_name"] = parcels_4s["atlas_name"].replace(
        {f"4S{n_4s}": f"Schaefer2018N{n_schaefer}n7"}
    )
    return parcels_4s.iloc[:n_schaefer, :]


def combine_atlases(schaefer_path, tian_path):
    schaefer_img = nib.load(schaefer_path)
    schaefer_data = np.round(schaefer_img.get_fdata()).astype(int)
    tian_img = nib.load(tian_path)
    tian_data = np.round(tian_img.get_fdata()).astype(int)
    combined_data = schaefer_data.copy()
    max_label = int(np.max(schaefer_data))
    tian_data_nonzero = tian_data > 0
    combined_data[tian_data_nonzero] = tian_data[tian_data_nonzero] + max_label
    # generate image with integer data type
    combined_img = nib.Nifti1Image(
        combined_data, schaefer_img.affine, schaefer_img.header, dtype=np.int16
    )
    return combined_img

In [4]:
FORCE = False
tian_schaefer_dir = Path(
    "/media/storage/yalab-dev/Tian2020MSA_v1.4/Tian2020MSA/3T/Subcortex-Only"
)
atlas_list = [
    f.name
    for f in tian_schaefer_dir.glob("MNIvolumetric/*MNI152NLin2009cAsym_1mm.nii.gz")
]

target_lut_format = (
    "Schaefer2018_{n_parcels}Parcels_7Networks_order_Tian_Subcortex_{scale}_label.txt"
)

tian_fillers = {
    "network_label": "Subcortex",
}


def extract_atlas_info(atlas_name: str):
    parts = atlas_name.split("_")
    n_schaefer = parts[1].replace("Parcels", "")
    scale = parts[-4]
    return n_schaefer, scale


new_atlases = []

for n_schaefer in np.arange(100, 1001, 100):
    for scale in [1, 2, 3, 4]:
        output_atlas_name = NAMING_SCHEME.format(
            n_parcels=n_schaefer, n_networks=7, scale=scale
        )
        new_atlases.append(output_atlas_name)
        print(f"Processing atlas: {output_atlas_name}")
        output_atlas_dir = (
            output_dir / "Schaefer2018Tian2020_atlases" / f"atlas-{output_atlas_name}"
        )
        output_atlas_dir.mkdir(parents=True, exist_ok=True)
        output_tsv = output_atlas_dir / f"atlas-{output_atlas_name}_dseg.tsv"
        output_nii = (
            output_atlas_dir
            / f"atlas-{output_atlas_name}_space-MNI152NLin2009cAsym_res-01_dseg.nii.gz"
        )
        if output_tsv.exists() and output_nii.exists() and not FORCE:
            print(f"Atlas {output_atlas_name} already exists. Skipping.")
            continue
        schaefer_nifti = tf.get(
            "MNI152NLin2009cAsym",
            atlas="Schaefer2018",
            desc=f"{n_schaefer}Parcels7Networks",
            resolution="01",
            extension=".nii.gz",
        )
        if not schaefer_nifti:
            print(
                f"Schaefer atlas with {n_schaefer} parcels not found in TemplateFlow. Skipping."
            )
            continue
        parcels_4s = load_lut(n_schaefer, scale)
        tian_nii_path = (
            tian_schaefer_dir / f"Tian_Subcortex_S{scale}_3T_2009cAsym_1mm.nii.gz"
        )
        tian_txt_path = tian_schaefer_dir / f"Tian_Subcortex_S{scale}_3T_label.txt"
        df_tian = pd.read_csv(tian_txt_path, header=None).rename(columns={0: "label"})
        # concat the two dataframes on the label column
        merged_df = pd.concat([parcels_4s, df_tian], axis=0).reset_index(drop=True)
        # Fill in missing values with the tian_fillers
        for key, value in tian_fillers.items():
            merged_df[key] = merged_df[key].fillna(value)
        merged_df["atlas_name"] = merged_df["atlas_name"].fillna(f"Tian2020S{scale}")
        merged_df["index"] = merged_df.index + 1
        # now create the combined atlas nifti

        combined_img = combine_atlases(schaefer_nifti, tian_nii_path)
        # save df to tsv
        merged_df.to_csv(output_tsv, sep="\t", index=False)

        nib.save(combined_img, output_nii)

Processing atlas: Schaefer2018N100n7Tian2020S1
Atlas Schaefer2018N100n7Tian2020S1 already exists. Skipping.
Processing atlas: Schaefer2018N100n7Tian2020S2
Atlas Schaefer2018N100n7Tian2020S2 already exists. Skipping.
Processing atlas: Schaefer2018N100n7Tian2020S3
Atlas Schaefer2018N100n7Tian2020S3 already exists. Skipping.
Processing atlas: Schaefer2018N100n7Tian2020S4
Atlas Schaefer2018N100n7Tian2020S4 already exists. Skipping.
Processing atlas: Schaefer2018N200n7Tian2020S1
Atlas Schaefer2018N200n7Tian2020S1 already exists. Skipping.
Processing atlas: Schaefer2018N200n7Tian2020S2
Atlas Schaefer2018N200n7Tian2020S2 already exists. Skipping.
Processing atlas: Schaefer2018N200n7Tian2020S3
Atlas Schaefer2018N200n7Tian2020S3 already exists. Skipping.
Processing atlas: Schaefer2018N200n7Tian2020S4
Atlas Schaefer2018N200n7Tian2020S4 already exists. Skipping.
Processing atlas: Schaefer2018N300n7Tian2020S1
Atlas Schaefer2018N300n7Tian2020S1 already exists. Skipping.
Processing atlas: Schaefer20

In [29]:
threshold = 0.95
# white matter atlas from the HCP
probabilistic_dir = Path("/media/storage/yalab-dev/voxelops/hcp1065_prob_coverage_nifti/prob")
probabilistic_4d: list = []


# first, generate a LUT for the probabilistic atlases
lut = pd.DataFrame(columns=["index","label", "atlas_name","hemisphere"])
atlas_name = "hcp1065_wm"
for i, prob_file in enumerate(sorted(probabilistic_dir.glob("*.nii.gz"))):
    label = "_".join(prob_file.name.split(".nii.gz")[0].split("_")[:-1])
    hemi = prob_file.name.split(".nii.gz")[0].split("_")[-1]
    lut.loc[i] = [i + 1, label, atlas_name, hemi]
    img = nib.load(prob_file)
    probabilistic_4d.append(img)
probabilistic_4d = np.stack([img.get_fdata() for img in probabilistic_4d], axis=-1)

combined_img = nib.Nifti1Image(probabilistic_4d, img.affine, img.header)
output_atlas_dir = output_dir / "hcp1065_wm_atlas" / f"atlas-{atlas_name}"




In [30]:
# atlas_3d = np.argmax(probabilistic_4d, axis=-1) + 1
# combined_img_3d = nib.Nifti1Image(atlas_3d, img.affine, img.header, dtype=np.int16)
output_tsv = output_atlas_dir / f"atlas-{atlas_name}_dseg.tsv"
output_nii = (output_atlas_dir / f"atlas-{atlas_name}_space-MNI152NLin2009cAsym_res-01_dseg.nii.gz")
output_atlas_dir.mkdir(parents=True, exist_ok=True)
lut.to_csv(output_tsv, sep="\t", index=False)
nib.save(combined_img, output_nii)

print("All atlases processed and saved successfully.")
print(f"Saved atlas to {output_atlas_dir}")

All atlases processed and saved successfully.
Saved atlas to /media/storage/yalab-dev/voxelops/hcp1065_wm_atlas/atlas-hcp1065_wm
